In [1]:
# ! pip install transformers
# ! pip install diffusers==0.20.2
# ! pip install accelerate

In [1]:
from tqdm.auto import tqdm
import os
from typing import List

import torch
from PIL import Image
import pandas as pd

from pipeline import AttendAndExcitePipeline
from utils import AttentionStore, register_attention_control

In [2]:
sd_version = "CompVis/stable-diffusion-v1-4"
# sd_version = "stabilityai/stable-diffusion-2-base"
token_indices = [2, 5]
seed = 42
output_path = "./results"
prompts_address = "../prompts/prompts.csv"
n_inference_steps = 50
guidance_scale = 7.5
max_iter_to_alter = 30
attention_res = 16
apply_loss = False
do_analysis = True
analysis_steps = list(range(0, 51, 1))
scale_factor = 20
scale_range = (1.0, 0.5)
save_cross_attention_maps = True

In [3]:
def load_model(sd_version):
    device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
    stable = AttendAndExcitePipeline.from_pretrained(sd_version).to(device)
    return stable

def run_on_prompt(prompt: List[str],
                  model: AttendAndExcitePipeline,
                  controller: AttentionStore,
                  generator: torch.Generator,
                  seed: int) -> Image.Image:
    if controller is not None:
        register_attention_control(model, controller)
    outputs, analysis_dict = model(prompt=prompt,
                                   attention_store=controller,
                                   indices_to_alter=token_indices,
                                   attention_res=attention_res,
                                   guidance_scale=guidance_scale,
                                   generator=generator,
                                   seed=seed,
                                   num_inference_steps=n_inference_steps,
                                   max_iter_to_alter=max_iter_to_alter,
                                   apply_loss=apply_loss,
                                   do_analysis=do_analysis,
                                   analysis_steps=analysis_steps,
                                   scale_factor=scale_factor,
                                   scale_range=scale_range,
                                   save_cross_attention_maps=save_cross_attention_maps,
                                   output_path=output_path)
    image = outputs.images[0]
    return image, analysis_dict

def save_image(image, output_path, prompt, seed):
    os.makedirs(f"{output_path}/{prompt}/{int(seed)}/", exist_ok=True)
    image.save(f"{output_path}/{prompt}/{int(seed)}/image.png")

def save_analysis(analysis_dict, output_path, prompt, seed):
    os.makedirs(f"{output_path}/{prompt}/{int(seed)}/", exist_ok=True)
    analysis_dict["step"] = analysis_steps
    analysis_df = pd.DataFrame(data=analysis_dict)
    analysis_df.to_csv(f"{output_path}/{prompt}/{int(seed)}/analysis.csv", index=False)

In [4]:
sd_model = load_model(sd_version)

text_encoder\model.safetensors not found


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
# %autoreload 2

if not os.path.exists(f'{output_path}/'):
    os.mkdir(f'{output_path}/')

prompts = pd.read_csv(f"{prompts_address}")["prompt"].tolist()
prompts = sorted(prompts)
# prompts = ['a dog and a cat', 'a bus and a car']
n_max = 1000000
for index, prompt in tqdm(enumerate(prompts)):
    if index  >= n_max:
        break
    print(f"{index+1}: {prompt}")
    g = torch.Generator('cuda').manual_seed(seed)
    controller = AttentionStore()
    image, analysis_dict = run_on_prompt(prompt=prompt,
                                         model=sd_model,
                                         controller=controller,
                                         generator=g,
                                         seed=seed)
    save_image(image=image, output_path=output_path, prompt=prompt, seed=seed)
    save_analysis(analysis_dict, output_path, prompt, seed)


0it [00:00, ?it/s]

1: a backpack and a banana


  0%|          | 0/50 [00:00<?, ?it/s]

c:\Users\user01\anaconda3\envs\attend_and_excite_2\lib\site-packages\diffusers\pipelines\stable_diffusion\pipeline_stable_diffusion.py:429: FutureWarning: The decode_latents method is deprecated and will be removed in a future version. Please use VaeImageProcessor instead
  warnings.warn(


2: a backpack and a bear


...

  0%|          | 0/50 [00:00<?, ?it/s]

2484: an umbrella and a window


  0%|          | 0/50 [00:00<?, ?it/s]

2485: an umbrella and a zebra


  0%|          | 0/50 [00:00<?, ?it/s]